In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [4]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [5]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [6]:

inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


In [7]:
# I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female

inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [8]:
inputs.columns[inputs.isna().any()]


Index(['Age'], dtype='object')

In [9]:
inputs.Age[:10]


0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [10]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train,y_train)
model.score(X_test,y_test)


0.7873134328358209

In [12]:
X_test[0:10]


,Pclass,Age,Fare,female
104,3,37.0,7.9250,0
550,1,17.0,110.8833,0
731,3,11.0,18.7875,0
350,3,23.0,9.2250,0
520,1,30.0,93.5000,1
847,3,35.0,7.8958,0
184,3,4.0,22.0250,1
603,3,44.0,8.0500,0
645,1,48.0,76.7292,0
742,1,21.0,262.3750,1


In [13]:
y_test[0:10]


104    0
550    1
731    0
350    0
520    1
847    0
184    1
603    0
645    1
742    1
Name: Survived, dtype: int64

In [14]:
model.predict(X_test[0:10])


array([0, 1, 0, 0, 1, 0, 1, 0, 0, 1], dtype=int64)

In [15]:
model.predict_proba(X_test[:10])


array([[9.67144581e-01, 3.28554186e-02],
       [1.59397124e-01, 8.40602876e-01],
       [9.47434920e-01, 5.25650799e-02],
       [9.62803270e-01, 3.71967300e-02],
       [1.54945737e-02, 9.84505426e-01],
       [9.67151410e-01, 3.28485899e-02],
       [2.40316691e-01, 7.59683309e-01],
       [9.65459222e-01, 3.45407778e-02],
       [5.52426009e-01, 4.47573991e-01],
       [3.73801243e-10, 1.00000000e+00]])

In [16]:
# Calculate the score using cross validation

from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.768     , 0.776     , 0.824     , 0.80645161, 0.72580645])